In [ ]:
# prompt: mount gg drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!rm -r /content/Prompt-Optimization-in-Hallucination-Detection
!git clone https://github.com/dunp7/Prompt-Optimization-in-Hallucination-Detection.git
!pip install -r /content/Prompt-Optimization-in-Hallucination-Detection/requirements.txt

rm: cannot remove '/content/Math_solving_using_LLM': No such file or directory
Cloning into 'Math_solving_using_LLM'...
remote: Enumerating objects: 311, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 311 (delta 59), reused 62 (delta 28), pack-reused 216 (from 1)
Receiving objects: 100% (311/311), 979.56 KiB | 12.40 MiB/s, done.
Resolving deltas: 100% (212/212), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.3 MB/s eta 0:00:0

In [ ]:
from datasets import load_from_disk
from huggingface_hub import login
import sys
import pandas as pd
import numpy as np

sys.path.append("/content/Prompt-Optimization-in-Hallucination-Detection")
sys.path.append("/content/Prompt-Optimization-in-Hallucination-Detection/my_utils")

# Extract Data

In [ ]:
### Datasets
datasets = ['svamps', 'gsm8k',]

### Generate answers LLM
models = {
    "falcon7b_instruct": "tiiuae/falcon-7b-instruct",
    "opt6_7b": "facebook/opt-6.7b",
    "opt_350m": "facebook/opt-350m",
    "opt1_3b": "facebook/opt-1.3b",
    "opt2_7b": "facebook/opt-2.7b",
    "gemma2_9b": "google/gemma-2-9b-it",
    "gemma2_2b": "google/gemma-2-2b-it",
    "llama2_7b": "meta-llama/Llama-2-7b-chat-hf",
    # "llama2_13b": "meta-llama/Llama-2-13b-chat-hf",
    "mistral7b_instruct": "mistralai/Mistral-7B-Instruct-v0.2",
}
### Prompts
prompts = {
    "default": None,
    "zero_shot_CoT": "Let’s think step by step.",
    "ite_ape": "We’ll approach this methodically, proceeding one step at a time.",
    "apo": "Carefully analyze the details and perform precise arithmetic operations step by step, ensuring to apply the correct mathematical principles for accurate calculations.",
    "pe2": "Carefully calculate step by step, considering all details, including unique counts and overlaps. Accurately apply arithmetic to find the numerical answer, ensuring logical operations are correctly followed."
}



In [ ]:
import os

base_path = "/content/drive/MyDrive/Gen_Data"


def take_data(base_path, dataset, models, prompts ,dataset_name):

  # Create variables
  data = {model: {} for model in models.keys()}


  # Process each model
  for model, _ in models.items():
      print(f"Processing model: {model}")

      data[model] = {prompt: [] for prompt in prompts.keys()}

      # Process each prompt
      for prompt in prompts.keys():
          # Retrieve data
          data_path = os.path.join(base_path, dataset, model, prompt, dataset_name)
          if os.path.exists(data_path):
              result = load_from_disk(data_path)
          else:
              print(f" No data found at {data_path}.")

          # Check if data is valid
          if data is not None:
              data[model][prompt].append(result)
          else:
              continue
  return data

# Data

In [ ]:
svamp_data = take_data(base_path, 'svamps', models ,prompts, 'svamp')
gsm8k_data = take_data(base_path, 'gsm8k', models, prompts, 'gsm8k')

Processing model: falcon7b_instruct
Processing model: opt6_7b
Processing model: opt_350m
Processing model: opt1_3b
Processing model: opt2_7b
Processing model: gemma2_9b
Processing model: gemma2_2b
Processing model: llama2_7b
Processing model: mistral7b_instruct
Processing model: falcon7b_instruct
Processing model: opt6_7b
Processing model: opt_350m
Processing model: opt1_3b
Processing model: opt2_7b
Processing model: gemma2_9b
Processing model: gemma2_2b
Processing model: llama2_7b
Processing model: mistral7b_instruct


In [ ]:
prompt_acc = pd.read_csv('/content/drive/MyDrive/Gen_Data/prompt_acc.csv')
prompt_acc

,Model,Prompt,Exact_Accuracy,f1_score,llm_score,dataset
0,falcon7b_instruct,default,0.04,0.04,0.02,svamp
1,falcon7b_instruct,zero_shot_CoT,0.06,0.06,0.04,svamp
2,falcon7b_instruct,ite_ape,0.06,0.06,0.06,svamp
3,falcon7b_instruct,apo,0.12,0.16,0.06,svamp
4,falcon7b_instruct,pe2,0.12,0.16,0.06,svamp
...,...,...,...,...,...,...
85,mistral7b_instruct,default,0.18,0.28,0.12,gsm8k
86,mistral7b_instruct,zero_shot_CoT,0.26,0.22,0.10,gsm8k
87,mistral7b_instruct,ite_ape,0.24,0.34,0.10,gsm8k
88,mistral7b_instruct,apo,0.20,0.24,0.08,gsm8k


# Analyze

In [ ]:
# # Specify the prompts of interest and their desired order
# prompts_order = ['default', 'zero_shot_CoT', 'ite_ape', 'apo', 'pe2']
# filtered_data = prompt_acc[prompt_acc['Prompt'].isin(prompts_order)]

# # Pivot the data to include all three scores for each prompt
# pivot_data_multi = filtered_data.pivot_table(
#     index=['dataset', 'Model'],
#     columns='Prompt',
#     values=['Exact_Accuracy', 'f1_score', 'llm_score'],
#     aggfunc='first'
# ).reset_index()

# # Flatten the multi-level columns for clarity
# pivot_data_multi.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col for col in pivot_data_multi.columns]

# # Reorder the columns to match the desired prompt order
# columns_order = ['dataset', 'Model']
# for metric in ['Exact_Accuracy', 'f1_score', 'llm_score']:
#     columns_order.extend([f"{metric}_{prompt}" for prompt in prompts_order])

# pivot_data_multi = pivot_data_multi[columns_order]

# # Save the result to an Excel file
# output_path = 'restructured_data_ordered.xlsx'
# pivot_data_multi.to_excel(output_path, index=False)

# print(f"Restructured data saved to {output_path}")

### Efficient Model

In [ ]:
# Average Accuracy of each model in each data for each prompt
prompt_acc['average_accuracy'] = prompt_acc[['Exact_Accuracy', 'f1_score', 'llm_score']].mean(axis=1)
average_accuracy = prompt_acc.groupby(['Model'])['average_accuracy'].mean().reset_index().sort_values(by='average_accuracy', ascending=False)

average_accuracy

,Model,average_accuracy
2,gemma2_9b,0.402000
4,mistral7b_instruct,0.296000
3,llama2_7b,0.231333
1,gemma2_2b,0.178667
7,opt6_7b,0.113333
6,opt2_7b,0.083333
8,opt_350m,0.076000
5,opt1_3b,0.070667
0,falcon7b_instruct,0.061333


## Prompt Importance

In [ ]:
prompt_performance = prompt_acc.groupby(['Prompt', 'Model'])[['Exact_Accuracy', 'f1_score', 'llm_score', 'average_accuracy']].mean().reset_index().sort_values(by='average_accuracy', ascending=False)
prompt_performance


,Prompt,Model,Exact_Accuracy,f1_score,llm_score,average_accuracy
20,ite_ape,gemma2_9b,0.46,0.72,0.12,0.433333
38,zero_shot_CoT,gemma2_9b,0.42,0.74,0.13,0.430000
2,apo,gemma2_9b,0.40,0.68,0.10,0.393333
11,default,gemma2_9b,0.38,0.61,0.19,0.393333
29,pe2,gemma2_9b,0.23,0.68,0.17,0.360000
22,ite_ape,mistral7b_instruct,0.35,0.46,0.12,0.310000
40,zero_shot_CoT,mistral7b_instruct,0.40,0.43,0.10,0.310000
30,pe2,llama2_7b,0.18,0.40,0.33,0.303333
31,pe2,mistral7b_instruct,0.33,0.44,0.10,0.290000
13,default,mistral7b_instruct,0.34,0.43,0.09,0.286667
